In [37]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("..")
import torch
from dnn_training.dataset_ak import *
from dnn_training.dataset_torch import *
import numpy as np

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
dataset_ak = loadDataset_ak("/grid_mnt/data_cms_upgrade/cuisset/supercls/alessandro_electrons/supercls-v13/superclsDumper_1.root")

In [8]:
d_zipped = zipDataset(dataset_ak)

607

In [19]:
ak.argmax(d_zipped.feature_seedPt, axis=-1)

<Array [217, 16, 22, 252, 130, ..., 696, 156, 88, 85, 63] type='100 * ?int64'>

In [23]:
d_zipped.seedTracksterIdx[ak.argmax(d_zipped.feature_seedPt, axis=-1, keepdims=True)][:, 0]

<Array [607, 631, 751, 2608, ..., 2334, 682, 634, 771] type='100 * ?uint32'>

In [34]:
ak.fill_none(d_zipped.seedTracksterIdx == d_zipped.seedTracksterIdx[ak.argmax(d_zipped.feature_seedPt, axis=-1, keepdims=True)][:, 0], [], 0)

<Array [[True, True, True, ..., False, False], ...] type='100 * var * bool'>

In [35]:
d_zipped[ak.fill_none(d_zipped.seedTracksterIdx == d_zipped.seedTracksterIdx[ak.argmax(d_zipped.feature_seedPt, axis=-1, keepdims=True)][:, 0], [], 0)]

<Array [[{...}, {...}, ..., {...}, {...}], ...] type='100 * var * inference...'>

In [38]:
tgt = makeTarget(selectSeedOnly(zipDataset(dataset_ak)))
tgt

<Array [[{...}, {...}, ..., {...}, {...}], ...] type='100 * var * inference...'>

In [43]:
ak.mean(tgt.genMatching)

0.015323368136408022

In [44]:
ak.mean(makeTarget(zipDataset(dataset_ak)).genMatching)

0.00031306191025168794

In [5]:
tmp = tgt[["feature_multi_en", "feature_multi_eta"]]
tmp

<Array [[{...}, {...}, ..., {...}, {...}], ...] type='100 * var * {feature_...'>

In [6]:
ak.type(tmp)

ArrayType(ListType(RecordType([NumpyType('float32'), NumpyType('float32')], ['feature_multi_en', 'feature_multi_eta'])), 100, None)

In [7]:
features = ['DeltaEtaBaryc', 'DeltaPhiBaryc', 'multi_en', 'multi_eta', 'multi_pt', 'seedEta','seedPhi','seedEn', 'seedPt', 'theta', 'theta_xz_seedFrame', 'theta_yz_seedFrame', 'theta_xy_cmsFrame', 'theta_yz_cmsFrame', 'theta_xz_cmsFrame', 'explVar', 'explVarRatio']

In [8]:
feats_ak = [ak.to_numpy(ak.flatten(tgt["feature_" + featName])) for featName in features]

In [9]:
np.stack(feats_ak, axis=1).shape

(1881417, 17)

In [10]:
torch.stack([torch.tensor(ar) for ar in feats_ak])

tensor([[ 1.4991e-01,  5.2284e-02,  1.6820e-01,  ...,  1.2566e-01,
         -2.0207e-02,  9.8886e-02],
        [ 1.6956e+00,  1.1760e+00,  5.6003e-01,  ..., -6.7385e-01,
          2.1588e+00,  1.8154e+00],
        [ 2.3841e+01,  2.6361e+01,  1.2826e+01,  ...,  2.2554e+00,
          2.4942e+00,  1.7655e+00],
        ...,
        [ 2.7881e-01,  5.5533e-02,  2.9864e-01,  ...,  2.4708e-02,
          1.1570e-02,  2.6077e-02],
        [ 4.4138e+01,  2.5784e+01,  3.4246e+01,  ...,  1.7453e+01,
          1.6417e+01,  3.8603e+00],
        [ 9.7236e-01,  9.3319e-01,  9.2995e-01,  ...,  9.7565e-01,
          9.5715e-01,  9.2568e-01]])

In [11]:
ds_t = makeTorchDataset(feats_ak, ak.to_numpy(ak.flatten(tgt.genMatching)))

In [27]:
torch.tensor(ak.to_numpy(ak.flatten(tgt.genMatching)), dtype=torch.float32)[23]

tensor(0.)

In [25]:
np.nonzero(ak.to_numpy(ak.flatten(tgt.genMatching)))

(array([     22,      44,      51,      63,      75,     115,     129,
            167,     179,     236,     326,     356,     358,     361,
            386,    7459,    7488,    7490,    7493,    7516,   13765,
          13788,   13802,   13820,   13898,   21862,   21874,   21892,
          21972,   26062,   26101,   26167,   26168,   26170,   42641,
          42643,   42991,   48027,   49901,   49913,   49915,   58122,
          58124,   73376,   73382,   73432,   73488,   73550,   73567,
          73665,   73692,   73729,   73738,   90850,   90881,   90899,
          90909,   90923,   90943,   90959,   90987,   91003,   91013,
          91126,  105071,  105131,  105155,  105217,  105266,  105312,
         105323,  105356,  105387,  105388,  105427,  105434,  105439,
         105454,  105462,  105463,  124667,  124684,  124735,  124786,
         124913,  124923,  124927,  146473,  146596,  166175,  166229,
         166248,  166284,  166286,  182436,  182441,  182445,  204979,
      

In [29]:
ds_t[0:50]["features"][0]

tensor([[ 1.4991e-01,  1.6956e+00,  2.3841e+01, -2.8661e+00,  2.7054e+00,
         -2.7162e+00,  6.6292e-01,  5.4562e+02,  6.1916e+01,  2.8193e-01,
          1.3713e-01,  1.3713e-01,  1.8231e+00,  5.3159e-02,  2.7881e-01,
          4.4138e+01,  9.7236e-01],
        [ 5.2284e-02,  1.1760e+00,  2.6361e+01, -2.7684e+00,  3.2958e+00,
         -2.7162e+00,  6.6292e-01,  5.4562e+02,  6.8218e+01,  1.1699e-01,
          7.7512e-03,  7.7512e-03,  6.5022e-01,  1.0302e-01,  5.5533e-02,
          2.5784e+01,  9.3319e-01],
        [ 1.6820e-01,  5.6003e-01,  1.2826e+01, -2.8844e+00,  1.4293e+00,
         -2.7162e+00,  6.6292e-01,  5.4562e+02,  6.0800e+01,  3.6202e-01,
          2.4707e-01,  2.4707e-01,  1.4768e+00,  2.1777e-01,  2.9864e-01,
          3.4246e+01,  9.2995e-01],
        [-8.6307e-02,  8.1876e-01,  1.6024e+01, -2.6298e+00,  2.2984e+00,
         -2.7162e+00,  6.6292e-01,  5.4562e+02,  7.8261e+01,  3.2822e-01,
          1.9353e-01,  1.9353e-01,  1.7767e+00,  9.9170e-02,  3.1675e-01,
    

In [18]:
ds_t[0:5]["genmatching"][0]

tensor([False, False, False, False, False])

In [19]:
loss = torch.nn.BCELoss()
loss(torch.full((5,), 0.5), ds_t[0:5]["genmatching"][0])

RuntimeError: Found dtype Bool but expected Float